In [1]:
from sklearn.feature_selection import RFECV, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import Imputer


from sklearn.metrics import roc_auc_score
from sklearn import linear_model

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
import os
import time

import load_data
import file_structure

from train_helper import RF_grid, RF_random_grid, Logreg_grid

def label_conversion(numpy_array):
    labels = numpy_array.tolist()
    labels_str = list()

    for i in range(len(labels)):

        if numpy_array[i] == 0:
            labels_str.append('non-codel')
        else:
            labels_str.append('codel')

    return labels_str


In [54]:
# modals
modal_list = ['FLAIR','T1', 'T1post','T2']
#study = 'CNN' # <== CHANGE HERE
# study_list = ['texture','TDA','CNN']
study_list = ['texture','TDA','CNN','cmb']
study_list = ['cmb']
ICC_filt = 0
if ICC_filt == 1:
    suffix = '_icc2'
else: 
    suffix = ''

for study in study_list:
    (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
    # load feature names and corr+rfe filtered feature names
    #if study == 'TDA':
    #    feature_names = np.load(data_dir + 'ph_feature_names.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir,'corr_rfe_ph_features_09.npy')).item()
    #elif study == 'texture':
    #    feature_names = np.load(data_dir + 'rfecv_tex_features_selected.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir, 'rfe_tex_features_selected.npy')).item()
    corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
    random_state_list = [2,4,6,8,10,12,14,16,18,20] 

    # For timing purpose
    time_list = []

    for fold in range(10):
        print('%s fold number: %d' % (study, fold))
        #t0 = time.time() python 2 vs python 3
        t0 = time.time()

        random_state = random_state_list[fold]

        fold_dir = os.path.join(splitdir,'fold_' + str(fold))

        #study_saved_model = data_dir_fold + 'saved_models/' + study + '_saved_model/'

        # load data
        (train_samples, train_features_pca, train_1p19q, train_age, 
         train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold,study,ICC_filt)
        (val_samples,val_features_pca, val_1p19q, val_age, 
         val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold,study,ICC_filt)

        feature_names = train_fnames

        # conver 1,0 into string: 'codel' and 'non-codel'
        train_1p19q_str = label_conversion(train_1p19q)
        val_1p19q_str = label_conversion(val_1p19q)

        # Create clinical variable df
        clinical_names = ['age', 'KPS', 'gender']
        train_clinical_df = pd.DataFrame(data = np.vstack((train_age, train_KPS, train_gender)).transpose(), 
                                            columns = clinical_names, index=train_1p19q_str)
        val_clinical_df = pd.DataFrame(data = np.vstack((val_age, val_KPS, val_gender)).transpose(), 
                                            columns = clinical_names, index=val_1p19q_str)

        # leave one group out for CV purpose later
        logo = LeaveOneGroupOut()

        for train_features, val_features, rfe_pca_flag in [(train_clinical_df, val_clinical_df,'clinical'),
                                                           (train_samples, val_samples,'rfe')
                                                          ]:

            if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, columns=feature_names, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, columns=feature_names, index=val_1p19q_str)

                # Remove Flatness and LeastAxis since they are all 0s for every sample
                drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                train_features_df = train_features_df.drop(drop_features, axis=1, errors = 'ignore')
                val_features_df = val_features_df.drop(drop_features, axis=1, errors = 'ignore')

                # obtain feature names to select from corr + RFE
                corr_rfe_selected = corr_rfe_selected_list[fold]

                # only the corr rfe selected features
                train_features_df = train_features_df[corr_rfe_selected]
                val_features_df = val_features_df[corr_rfe_selected]

            elif rfe_pca_flag == 'pca': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
            elif rfe_pca_flag == 'clinical': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)

            # combine train and val for CV purpose
            train_val_features_df = pd.concat([train_features_df, val_features_df], axis=0)
            train_val_clinical_df = pd.concat([train_clinical_df, val_clinical_df], axis=0)
            train_val_1p19q = np.append(train_1p19q, val_1p19q)
            train_val_1p19q_str = train_1p19q_str + val_1p19q_str

            # combine the membership as well for CV purpose
            #train_val_group_membership = train_group_membership + val_group_membership
            train_val_id = np.append(train_id, val_id)

            # features + clinical 
            train_val_all_df = pd.concat([train_val_features_df, train_val_clinical_df], axis=1)



            for sample_case in ['features', 'all']: #features without clinical vs features + clinical
                # group kfold split (initially unused, donnie did leave one out)
                # cross validation groups were too small, and it was causing errors
                #if sample_case == 'features':
                #elif sample_case == 'all':
                if (rfe_pca_flag == 'clinical' and sample_case != 'features'):
                    continue # only run clinical once through

                for classifier in ['rf','logreg']:
                    print(sample_case,classifier,rfe_pca_flag)
                    if classifier == 'rf':
                        estimator_instance = RandomForestClassifier() 
                        estimator_grid = RF_random_grid()

                        n_iter = 100
                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)

                        elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)
                            estimator.fit(train_val_all_df, train_val_1p19q)

                    elif classifier == 'logreg':
                        estimator_instance = LogisticRegression()
                        estimator_grid = Logreg_grid()

                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)

                        elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_all_df, train_val_1p19q)

                    best_estimator = estimator.best_estimator_.__dict__
                    best_score = estimator.best_score_

                    # save models and train score

    #                if rfe_pca_flag == 'rfe': # corr + rfe
    #                    estimator_name = classifier + '_' + sample_case +'_rfe_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier + '_' + sample_case +'_rfe_' + str(fold) + '_best_cv_score.npy'

    #                elif rfe_pca_flag == 'pca': # pca
    #                    estimator_name = classifier + '_' + sample_case +'_pca_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier + '_' + sample_case +'_pca_' + str(fold) + '_best_cv_score.npy'
    #                elif rfe_pca_flag == 'clinical': # pca
    #                    estimator_name = classifier '_' + sample_case + '_clinical_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier'_' + sample_case + '_clinical_' + str(fold) + '_best_cv_score.npy'

                    estimator_name = (classifier + '_' + sample_case + '_'
                                      + rfe_pca_flag + '_' + str(fold) +suffix + '.npy')
                    estimator_score_name =(classifier +  '_' + sample_case + '_'+ rfe_pca_flag 
                                           +'_' + str(fold) + '_best_cv_score'+suffix+'.npy')

                    np.save(os.path.join(model_dir,estimator_name), best_estimator)
                    np.save(os.path.join(model_dir,estimator_score_name), best_score)

        t1 = time.time()
        time_list.append((t1-t0)/60)
        print('%s FOLD: %d DONE WITH TIME TAKEN (IN MIN): %g' % (study, fold, (t1-t0)/60))


cmb fold number: 0
features rf clinical
Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [57]:
# modals
modal_list = ['FLAIR','T1', 'T1post','T2']
#study = 'CNN' # <== CHANGE HERE
# study_list = ['texture','TDA','CNN']
study_list = ['texture','TDA','CNN','cmb']
study_list = ['texture']
ICC_filt = 0
if ICC_filt == 1:
    suffix = '_icc2'
else: 
    suffix = ''

for study in study_list:
    (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
    # load feature names and corr+rfe filtered feature names
    #if study == 'TDA':
    #    feature_names = np.load(data_dir + 'ph_feature_names.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir,'corr_rfe_ph_features_09.npy')).item()
    #elif study == 'texture':
    #    feature_names = np.load(data_dir + 'rfecv_tex_features_selected.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir, 'rfe_tex_features_selected.npy')).item()
    corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
    random_state_list = [2,4,6,8,10,12,14,16,18,20] 

    # For timing purpose
    time_list = []
#     if 1:
#         fold = 0
    for fold in range(10):
        print('%s fold number: %d' % (study, fold))
        #t0 = time.time() python 2 vs python 3
        t0 = time.time()

        random_state = random_state_list[fold]

        fold_dir = os.path.join(splitdir,'fold_' + str(fold))

        #study_saved_model = data_dir_fold + 'saved_models/' + study + '_saved_model/'

        # load data
        (train_samples, train_features_pca, train_1p19q, train_age, 
         train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold,study,ICC_filt)
        (val_samples,val_features_pca, val_1p19q, val_age, 
         val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold,study,ICC_filt)

        feature_names = train_fnames

        # conver 1,0 into string: 'codel' and 'non-codel'
        train_1p19q_str = label_conversion(train_1p19q)
        val_1p19q_str = label_conversion(val_1p19q)

        # Create clinical variable df
        clinical_names = ['age', 'KPS', 'gender']
        train_clinical_df = pd.DataFrame(data = np.vstack((train_age, train_KPS, train_gender)).transpose(), 
                                            columns = clinical_names, index=train_1p19q_str)
        train_clin_by_pt_df = train_clinical_df.assign(pid = pd.Series(train_id).values)
        train_clin_by_pt_df = train_clin_by_pt_df.groupby('pid').mean()
        val_clinical_df = pd.DataFrame(data = np.vstack((val_age, val_KPS, val_gender)).transpose(), 
                                            columns = clinical_names, index=val_1p19q_str)
        val_clin_by_pt_df = val_clinical_df.assign(pid = pd.Series(val_id).values)
        val_clin_by_pt_df = val_clin_by_pt_df.groupby('pid').mean()
        
        for train_features, val_features, rfe_pca_flag in [(train_clin_by_pt_df, val_clin_by_pt_df,'clin_ppt')
                                                          ]:

            if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, columns=feature_names, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, columns=feature_names, index=val_1p19q_str)

                # Remove Flatness and LeastAxis since they are all 0s for every sample
                drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                train_features_df = train_features_df.drop(drop_features, axis=1, errors = 'ignore')
                val_features_df = val_features_df.drop(drop_features, axis=1, errors = 'ignore')

                # obtain feature names to select from corr + RFE
                corr_rfe_selected = corr_rfe_selected_list[fold]

                # only the corr rfe selected features
                train_features_df = train_features_df[corr_rfe_selected]
                val_features_df = val_features_df[corr_rfe_selected]

            elif rfe_pca_flag == 'pca': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
            elif rfe_pca_flag == 'clinical':  #clinical variables
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
            elif rfe_pca_flag == 'clin_ppt':  #clinical per patient variables
                # create the pandas dataframes
                train_1p19q_df = pd.DataFrame(data=np.vstack((train_1p19q, train_id)).T, 
                                              columns=('1p19q','pid'), index=train_1p19q_str)
                val_1p19q_df = pd.DataFrame(data=np.vstack((val_1p19q, val_id)).T, 
                                              columns=('1p19q','pid'), index=val_1p19q_str)
                ct_1p19q = train_1p19q_df.groupby('pid')['1p19q'].mean().values
                cv_1p19q = val_1p19q_df.groupby('pid')['1p19q'].mean().values
                train_val_1p19q = np.append(ct_1p19q,cv_1p19q)
                train_val_1p19q_str = label_conversion(train_val_1p19q)
                
                train_features_df = pd.DataFrame(data=train_features.values, columns=clinical_names, index=label_conversion(ct_1p19q))
                val_features_df = pd.DataFrame(data=val_features.values, columns=clinical_names, index=label_conversion(cv_1p19q))

            # combine train and val for CV purpose
            train_val_features_df = pd.concat([train_features_df, val_features_df], axis=0)
            train_val_clinical_df = pd.concat([train_clinical_df, val_clinical_df], axis=0)

            # combine the membership as well for CV purpose
            #train_val_group_membership = train_group_membership + val_group_membership
            train_val_id = np.append(train_id, val_id)

            
            if rfe_pca_flag == 'clin_ppt': #clinical per patient variables
                train_val_id = np.append(train_clin_by_pt_df.index.values, val_clin_by_pt_df.index.values)
            else:
                train_val_1p19q = np.append(train_1p19q, val_1p19q)
                train_val_1p19q_str = train_1p19q_str + val_1p19q_str
                # features + clinical 
                train_val_all_df = pd.concat([train_val_features_df, train_val_clinical_df], axis=1)


            for sample_case in ['features', 'all']: #features without clinical vs features + clinical
                # group kfold split (initially unused, donnie did leave one out)
                # cross validation groups were too small, and it was causing errors
                #if sample_case == 'features':
                #elif sample_case == 'all':
                if ((rfe_pca_flag == 'clinical' or rfe_pca_flag == 'clin_ppt') and sample_case != 'features'):
                    continue # only run clinical once through

                for classifier in ['rf','logreg']:
                    print(sample_case,classifier,rfe_pca_flag)
                    if classifier == 'rf':
                        estimator_instance = RandomForestClassifier() 
                        estimator_grid = RF_random_grid()

                        n_iter = 100
                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            if rfe_pca_flag == 'clin_ppt':
                                gkf = StratifiedKFold(n_splits=5)
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)

                        elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)
                            estimator.fit(train_val_all_df, train_val_1p19q)

                    elif classifier == 'logreg':
                        estimator_instance = LogisticRegression()
                        estimator_grid = Logreg_grid()

                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            if rfe_pca_flag == 'clin_ppt':
                                gkf = StratifiedKFold(n_splits=5)
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)
                            
                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)
                    elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_all_df, train_val_1p19q)

                    best_estimator = estimator.best_estimator_.__dict__
                    best_score = estimator.best_score_
                    
                    estimator_name = (classifier + '_' + sample_case + '_'
                                      + rfe_pca_flag + '_' + str(fold) +suffix + '.npy')
                    estimator_score_name =(classifier +  '_' + sample_case + '_'+ rfe_pca_flag 
                                           +'_' + str(fold) + '_best_cv_score'+suffix+'.npy')

                    np.save(os.path.join(model_dir,estimator_name), best_estimator)
                    np.save(os.path.join(model_dir,estimator_score_name), best_score)

        t1 = time.time()
        time_list.append((t1-t0)/60)
        print('%s FOLD: %d DONE WITH TIME TAKEN (IN MIN): %g' % (study, fold, (t1-t0)/60))


texture fold number: 0
features rf clin_ppt
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.9min finished


features logreg clin_ppt
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  75 out of  90 | elapsed:    4.4s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    4.4s finished


texture FOLD: 0 DONE WITH TIME TAKEN (IN MIN): 4.04395
texture fold number: 1
features rf clin_ppt
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    5.8s finished


texture FOLD: 8 DONE WITH TIME TAKEN (IN MIN): 4.21357
texture fold number: 9
features rf clin_ppt
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.8min finished


features logreg clin_ppt
Fitting 5 folds for each of 18 candidates, totalling 90 fits
texture FOLD: 9 DONE WITH TIME TAKEN (IN MIN): 1.88663


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  75 out of  90 | elapsed:    3.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    3.8s finished


In [58]:
print(train_val_features_df, train_val_1p19q)
print(train_1p19q.shape,train_val_1p19q.shape,train_val_id.shape)

            age    KPS  gender
non-codel  56.0   80.0     1.0
non-codel  61.0   80.0     1.0
non-codel  18.0   80.0     1.0
non-codel  33.0  100.0     1.0
non-codel  54.0  100.0     0.0
non-codel  74.0   80.0     1.0
non-codel  61.0   60.0     0.0
non-codel  44.0   80.0     1.0
non-codel  57.0   80.0     0.0
non-codel  31.0   80.0     1.0
non-codel  70.0   80.0     0.0
non-codel  63.0   80.0     0.0
non-codel  63.0   80.0     1.0
non-codel  45.0  100.0     1.0
non-codel  51.0   80.0     0.0
non-codel  84.0   80.0     1.0
non-codel  54.0   80.0     0.0
non-codel  43.0   80.0     0.0
non-codel  53.0   80.0     1.0
non-codel  73.0   80.0     0.0
non-codel  34.0   80.0     0.0
non-codel  63.0   80.0     0.0
non-codel  54.0  100.0     0.0
non-codel  69.0   60.0     0.0
non-codel  71.0  100.0     0.0
non-codel  62.0   80.0     0.0
non-codel  58.0  100.0     0.0
non-codel  57.0   90.0     0.0
non-codel  71.0   80.0     0.0
non-codel  60.0   80.0     0.0
...         ...    ...     ...
non-code

In [63]:
study_list = ['texture','TDA','CNN','cmb']
for study in study_list:
    (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
    # load feature names and corr+rfe filtered feature names
    #if study == 'TDA':
    #    feature_names = np.load(data_dir + 'ph_feature_names.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir,'corr_rfe_ph_features_09.npy')).item()
    #elif study == 'texture':
    #    feature_names = np.load(data_dir + 'rfecv_tex_features_selected.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir, 'rfe_tex_features_selected.npy')).item()
    corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
    random_state_list = [2,4,6,8,10,12,14,16,18,20] 

    # For timing purpose
    time_list = []

    for fold in range(10):
        print('%s fold number: %d' % (study, fold))
        #t0 = time.time() python 2 vs python 3
        t0 = time.time()

        random_state = random_state_list[fold]

        fold_dir = os.path.join(splitdir,'fold_' + str(fold))

        #study_saved_model = data_dir_fold + 'saved_models/' + study + '_saved_model/'

        # load data
        (train_samples, train_features_pca, train_1p19q, train_age, 
         train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold,study,ICC_filt)
        (val_samples,val_features_pca, val_1p19q, val_age, 
         val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold,study,ICC_filt)
        
        feature_names = train_fnames
        # add PID to DF
        test_samples = np.hstack((np.expand_dims(test_id,1),test_samples))
        feature_names  =np.hstack((['pid'],feature_names))
        
        
        # conver 1,0 into string: 'codel' and 'non-codel'
        train_1p19q_str = label_conversion(train_1p19q)
        val_1p19q_str = label_conversion(val_1p19q)

        # Create clinical variable df
        clinical_names = ['age', 'KPS', 'gender']
        train_clinical_df = pd.DataFrame(data = np.vstack((train_age, train_KPS, train_gender)).transpose(), 
                                            columns = clinical_names, index=train_1p19q_str)
        val_clinical_df = pd.DataFrame(data = np.vstack((val_age, val_KPS, val_gender)).transpose(), 
                                            columns = clinical_names, index=val_1p19q_str)

        # leave one group out for CV purpose later
        logo = LeaveOneGroupOut()

        for train_features, val_features, rfe_pca_flag in [(train_features_pca, val_features_pca,'pca'),
                                                           (train_clinical_df, val_clinical_df,'clinical'),
                                                           (train_samples, val_samples,'rfe')
                                                          ]:

            if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, columns=feature_names, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, columns=feature_names, index=val_1p19q_str)

                # Remove Flatness and LeastAxis since they are all 0s for every sample
                drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                train_features_df = train_features_df.drop(drop_features, axis=1, errors = 'ignore')
                val_features_df = val_features_df.drop(drop_features, axis=1, errors = 'ignore')

                # obtain feature names to select from corr + RFE
                corr_rfe_selected = corr_rfe_selected_list[fold]

                # only the corr rfe selected features
                train_features_df = train_features_df[corr_rfe_selected]
                val_features_df = val_features_df[corr_rfe_selected]

            elif rfe_pca_flag == 'pca': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
            elif rfe_pca_flag == 'clinical': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)

            # combine train and val for CV purpose
            train_val_features_df = pd.concat([train_features_df, val_features_df], axis=0)
            train_val_clinical_df = pd.concat([train_clinical_df, val_clinical_df], axis=0)
            train_val_1p19q = np.append(train_1p19q, val_1p19q)
            train_val_1p19q_str = train_1p19q_str + val_1p19q_str

            # combine the membership as well for CV purpose
            #train_val_group_membership = train_group_membership + val_group_membership
            train_val_id = np.append(train_id, val_id)

            # features + clinical 
            train_val_all_df = pd.concat([train_val_features_df, train_val_clinical_df], axis=1)
            # save to excel
            output_name = ('training'+'_'+study + '_' + rfe_pca_flag + '_' + str(fold) +suffix + '.csv')
            train_val_features_df.to_csv(os.path.join(out_dir,output_name))
    

texture fold number: 0
texture fold number: 1
texture fold number: 2
texture fold number: 3
texture fold number: 4
texture fold number: 5
texture fold number: 6
texture fold number: 7
texture fold number: 8
texture fold number: 9
TDA fold number: 0
TDA fold number: 1
TDA fold number: 2
TDA fold number: 3
TDA fold number: 4
TDA fold number: 5
TDA fold number: 6
TDA fold number: 7
TDA fold number: 8
TDA fold number: 9
CNN fold number: 0
CNN fold number: 1
CNN fold number: 2
CNN fold number: 3
CNN fold number: 4
CNN fold number: 5
CNN fold number: 6
CNN fold number: 7
CNN fold number: 8
CNN fold number: 9
cmb fold number: 0
cmb fold number: 1
cmb fold number: 2
cmb fold number: 3
cmb fold number: 4
cmb fold number: 5
cmb fold number: 6
cmb fold number: 7
cmb fold number: 8
cmb fold number: 9


In [14]:
study_list = ['texture','TDA','CNN','cmb']
ICC_filt = 0
if ICC_filt == 1:
    suffix = '_icc2'
else: 
    suffix = ''

for study in study_list:

    (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
    corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
    random_state_list = [2,4,6,8,10,12,14,16,18,20] 


    res_arr = []
    for fold in range(10):
        result = {}
        print('%s fold number: %d' % (study, fold))
        #(train_samples, train_pca , train_1p19q, train_age, train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold)
        #(val_samples, val_pca , val_1p19q, val_age, val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold)
        (test_samples, test_pca , test_1p19q, test_age,
         test_KPS, test_gender, test_id, feature_names)= load_data.load_data('test',fold, study,ICC_filt)
        
        # add PID to DF
        test_samples = np.hstack((np.expand_dims(test_id,1),test_samples))
        feature_names  =np.hstack((['pid'],feature_names))
        
        
        
        fold_dir = os.path.join(splitdir,'fold_' + str(fold))

        # convert 1,0 into string: 'codel' and 'non-codel'
        test_1p19q_str = label_conversion(test_1p19q)

        # create the pandas dataframes
        data_test_imaging = pd.DataFrame(data=test_samples, columns=feature_names, index=test_1p19q_str)

        clinical_names = ['age', 'KPS', 'gender']
        test_clinical_df = pd.DataFrame(data = np.vstack((test_age, test_KPS, test_gender)).transpose(),
                                        columns = clinical_names, index=test_1p19q_str)
        test_clin_by_pt_df = test_clinical_df.assign(pid = pd.Series(test_id).values)
        test_clin_by_pt_df = test_clin_by_pt_df.groupby('pid').mean()
        #data_test_age = pd.DataFrame(data = np.hstack((test_age)), columns = ['age'], index=test_1p19q_str)


#         for test_features, rfe_pca_flag in [(test_clin_by_pt_df,'clin_ppt')]:
        for test_features, rfe_pca_flag in [(test_clinical_df,'clinical'),
                                            (test_samples,'rfe'),
                                            (test_pca,'pca')]:

            if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                # create the pandas dataframes
                test_features_df = pd.DataFrame(data=test_features, columns=feature_names, index=test_1p19q_str)

                # Remove Flatness and LeastAxis since they are all 0s for every sample
                drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                test_features_df = test_features_df.drop(drop_features, axis=1 , errors = 'ignore')

                # obtain feature names to select from corr + RFE
                corr_rfe_selected = corr_rfe_selected_list[fold]

                # only the corr rfe selected features
                test_features_df = test_features_df[corr_rfe_selected]

            elif rfe_pca_flag == 'pca': #PCA features
                # create the pandas dataframes            
                test_features_df = pd.DataFrame(data=test_features, index=test_1p19q_str)

            elif rfe_pca_flag == 'clinical': #PCA features
                if study != 'cmb':
                    continue
                # create the pandas dataframes            
                test_features_df = pd.DataFrame(data=test_features, index=test_1p19q_str)
            
            output_name = ('testing'+'_'+study+'_' + rfe_pca_flag + '_' + str(fold) +suffix + '.csv')
            
            test_features_df.to_csv(os.path.join(out_dir,output_name))

texture fold number: 0
texture fold number: 1
texture fold number: 2
texture fold number: 3
texture fold number: 4
texture fold number: 5
texture fold number: 6
texture fold number: 7
texture fold number: 8
texture fold number: 9
TDA fold number: 0
TDA fold number: 1
TDA fold number: 2
TDA fold number: 3
TDA fold number: 4
TDA fold number: 5
TDA fold number: 6
TDA fold number: 7
TDA fold number: 8
TDA fold number: 9
CNN fold number: 0
CNN fold number: 1
CNN fold number: 2
CNN fold number: 3
CNN fold number: 4
CNN fold number: 5
CNN fold number: 6
CNN fold number: 7
CNN fold number: 8
CNN fold number: 9
cmb fold number: 0
cmb fold number: 1
cmb fold number: 2
cmb fold number: 3
cmb fold number: 4
cmb fold number: 5
cmb fold number: 6
cmb fold number: 7
cmb fold number: 8
cmb fold number: 9


In [10]:
print(test_id.shape)
print(test_samples.shape)

(112,)
(112, 239)


In [3]:

# modals
modal_list = ['FLAIR','T1', 'T1post','T2']
#study = 'CNN' # <== CHANGE HERE
# study_list = ['texture','TDA','CNN']
study_list = ['texture','TDA','CNN','cmb']
study_list = ['cmb']
ICC_filt = 0

study_list = ['texture','TDA']
ICC_filt = 1

# study_list = ['texture','TDA','CNN','cmb']
# ICC_filt = 0

if ICC_filt == 1:
    suffix = '_icc2'
else: 
    suffix = ''

for study in study_list:
    (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
    # load feature names and corr+rfe filtered feature names
    #if study == 'TDA':
    #    feature_names = np.load(data_dir + 'ph_feature_names.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir,'corr_rfe_ph_features_09.npy')).item()
    #elif study == 'texture':
    #    feature_names = np.load(data_dir + 'rfecv_tex_features_selected.npy')
    #    corr_rfe_selected_list = np.load(os.path.join(out_dir, 'rfe_tex_features_selected.npy')).item()
    corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
    random_state_list = [2,4,6,8,10,12,14,16,18,20] 

    # For timing purpose
    time_list = []

    for fold in range(10):
        print('%s fold number: %d' % (study, fold))
        #t0 = time.time() python 2 vs python 3
        t0 = time.time()

        random_state = random_state_list[fold]

        fold_dir = os.path.join(splitdir,'fold_' + str(fold))

        #study_saved_model = data_dir_fold + 'saved_models/' + study + '_saved_model/'

        # load data
        (train_samples, train_features_pca, train_1p19q, train_age, 
         train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold,study,ICC_filt)
        (val_samples,val_features_pca, val_1p19q, val_age, 
         val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold,study,ICC_filt)

        feature_names = train_fnames

        # conver 1,0 into string: 'codel' and 'non-codel'
        train_1p19q_str = label_conversion(train_1p19q)
        val_1p19q_str = label_conversion(val_1p19q)

        # Create clinical variable df
        clinical_names = ['age', 'KPS', 'gender']
        train_clinical_df = pd.DataFrame(data = np.vstack((train_age, train_KPS, train_gender)).transpose(), 
                                            columns = clinical_names, index=train_1p19q_str)
        val_clinical_df = pd.DataFrame(data = np.vstack((val_age, val_KPS, val_gender)).transpose(), 
                                            columns = clinical_names, index=val_1p19q_str)

        # leave one group out for CV purpose later
        logo = LeaveOneGroupOut()

        for train_features, val_features, rfe_pca_flag in [(train_features_pca, val_features_pca,'pca'),
                                                           (train_clinical_df, val_clinical_df,'clinical'),
                                                           (train_samples, val_samples,'rfe')
                                                          ]:

            if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, columns=feature_names, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, columns=feature_names, index=val_1p19q_str)

                # Remove Flatness and LeastAxis since they are all 0s for every sample
                drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                train_features_df = train_features_df.drop(drop_features, axis=1, errors = 'ignore')
                val_features_df = val_features_df.drop(drop_features, axis=1, errors = 'ignore')

                # obtain feature names to select from corr + RFE
                corr_rfe_selected = corr_rfe_selected_list[fold]

                # only the corr rfe selected features
                train_features_df = train_features_df[corr_rfe_selected]
                val_features_df = val_features_df[corr_rfe_selected]

            elif rfe_pca_flag == 'pca': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
            elif rfe_pca_flag == 'clinical': #PCA features
                # create the pandas dataframes
                train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)

            # combine train and val for CV purpose
            train_val_features_df = pd.concat([train_features_df, val_features_df], axis=0)
            train_val_clinical_df = pd.concat([train_clinical_df, val_clinical_df], axis=0)
            train_val_1p19q = np.append(train_1p19q, val_1p19q)
            train_val_1p19q_str = train_1p19q_str + val_1p19q_str

            # combine the membership as well for CV purpose
            #train_val_group_membership = train_group_membership + val_group_membership
            train_val_id = np.append(train_id, val_id)

            # features + clinical 
            train_val_all_df = pd.concat([train_val_features_df, train_val_clinical_df], axis=1)



            for sample_case in ['features', 'all']: #features without clinical vs features + clinical
                # group kfold split (initially unused, donnie did leave one out)
                # cross validation groups were too small, and it was causing errors
                #if sample_case == 'features':
                #elif sample_case == 'all':
                if (rfe_pca_flag == 'clinical' and sample_case != 'features'):
                    continue # only run clinical once through

                for classifier in ['rf','logreg']:
                    print(sample_case,classifier,rfe_pca_flag)
                    if classifier == 'rf':
                        estimator_instance = RandomForestClassifier() 
                        estimator_grid = RF_random_grid()

                        n_iter = 100
                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)

                        elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = RandomizedSearchCV(
                                estimator = estimator_instance, 
                                param_distributions = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                random_state = random_state,
                                n_iter = n_iter,
                                refit = True,
                                n_jobs = -1)
                            estimator.fit(train_val_all_df, train_val_1p19q)

                    elif classifier == 'logreg':
                        estimator_instance = LogisticRegression()
                        estimator_grid = Logreg_grid()

                        if sample_case == 'features':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_features_df, train_val_1p19q)

                        elif sample_case == 'all':
                            gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                            gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                            estimator = GridSearchCV(
                                estimator = estimator_instance, 
                                param_grid = estimator_grid,
                                scoring = 'roc_auc',
                                #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                cv = gkfSp,
                                verbose=2,
                                refit = True,
                                n_jobs = -1)

                            estimator.fit(train_val_all_df, train_val_1p19q)

                    best_estimator = estimator.best_estimator_.__dict__
                    best_score = estimator.best_score_

                    # save models and train score

    #                if rfe_pca_flag == 'rfe': # corr + rfe
    #                    estimator_name = classifier + '_' + sample_case +'_rfe_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier + '_' + sample_case +'_rfe_' + str(fold) + '_best_cv_score.npy'

    #                elif rfe_pca_flag == 'pca': # pca
    #                    estimator_name = classifier + '_' + sample_case +'_pca_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier + '_' + sample_case +'_pca_' + str(fold) + '_best_cv_score.npy'
    #                elif rfe_pca_flag == 'clinical': # pca
    #                    estimator_name = classifier '_' + sample_case + '_clinical_' + str(fold) + '.npy'
    #                    estimator_score_name = classifier'_' + sample_case + '_clinical_' + str(fold) + '_best_cv_score.npy'

                    estimator_name = (classifier + '_' + sample_case + '_'
                                      + rfe_pca_flag + '_' + str(fold) +suffix + '.npy')
                    estimator_score_name =(classifier +  '_' + sample_case + '_'+ rfe_pca_flag 
                                           +'_' + str(fold) + '_best_cv_score'+suffix+'.npy')

                    np.save(os.path.join(model_dir,estimator_name), best_estimator)
                    np.save(os.path.join(model_dir,estimator_score_name), best_score)

        t1 = time.time()
        time_list.append((t1-t0)/60)
        print('%s FOLD: %d DONE WITH TIME TAKEN (IN MIN): %g' % (study, fold, (t1-t0)/60))


texture fold number: 0
features rf pca
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.9min finished


features logreg pca
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  75 out of  90 | elapsed:    3.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    3.9s finished


all rf pca
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

all rf rfe
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.9min finished


all logreg rfe
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  75 out of  90 | elapsed:    3.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    3.7s finished


TDA FOLD: 9 DONE WITH TIME TAKEN (IN MIN): 10.3337


In [72]:
print(sample_case,classifier,rfe_pca_flag)
print(train_val_features_df)
print(train_features_pca)
dataset = 'train'
mode = study
pca_name = os.path.join(fold_dir, '%s_%s_pca%s.npy' % (dataset,mode[0:3].lower(),suffix))
print(pca_name)

features rf pca
Empty DataFrame
Columns: []
Index: [non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, non-codel, no